### Basic Article Enrichment and Relationship Stucture

In [1]:
import re
import networkx as nx
import matplotlib.colors as clr
from utils.graph import Graph_Viz_Engine
from utils.io import read_in


In [2]:
article_relations = './data/outbound-requests-by-domain-2.json'
relations = read_in(article_relations)

In [3]:
def extract_domain(url: str) -> str:
    
    pattern = r"(((([A-Za-z0-9]+){1,63}\.)|(([A-Za-z0-9]+(\-)+[A-Za-z0-9]+){1,63}\.))+){1,255}"
    domain = re.search(pattern, url)

    return domain.group().replace('www', '').replace('.', '')

In [4]:
n_relations = {}

for idx, key in enumerate(relations.keys()):
    n_key= extract_domain(key)

    n_relations[n_key] = relations[key]

In [5]:
t_blob = n_relations['nytimes']['content']

In [6]:
# analyze these text together and make a graph

In [7]:
from textblob import TextBlob
from utils.stopwords import stopwords
from string import punctuation


In [8]:
nyt_blob = TextBlob(t_blob)


In [ ]:
stopwords(additional=punctuation.split())

In [9]:
polarities = []

for sentence in nyt_blob.sentences:
    polarities.append(sentence.sentiment.polarity)

avg_polarity_of_sentences = sum(polarities) / len(polarities)

In [27]:
filtered_words = [ word for word in sorted(nyt_blob.word_counts, key=nyt_blob.word_counts.get, reverse=True) if word not in stopwords(additional=[p for p in punctuation]) and len(word) >= 3]

In [28]:
filtered_words

['russian',
 'ukraine',
 'russia',
 'putin',
 'dugina',
 'war',
 'dugin',
 'kremlin',
 'moscow',
 'attack',
 'sunday',
 'political',
 'television',
 'saturday',
 'image',
 'credit',
 'social',
 'military',
 'kyiv',
 'suburb',
 'invasion',
 'ukrainian',
 'wrote',
 'media',
 'car',
 'bombing',
 'daughter',
 'president',
 'killing',
 'daria',
 'imperialist',
 'escalate',
 'role',
 'pro-kremlin',
 'ultranationalist',
 'death',
 'captured',
 'describing',
 'brazen',
 'elite',
 'hawkish',
 'commentator',
 'aleksandr',
 'long',
 'assault',
 'occurred',
 'terrorist',
 'intended',
 'news',
 'evidence',
 'adviser',
 'played',
 'deep',
 'reporting',
 'global',
 'understand',
 'support',
 'attacks',
 'photo',
 'reuters',
 'calls',
 'enemy',
 'apachev',
 'nationalist',
 'critics',
 'proponents',
 'campaign',
 'capital',
 'central',
 'york',
 'times',
 'visible',
 'forces',
 'step',
 'government',
 'buildings',
 'supporters',
 'leader',
 'dangerous',
 'years',
 'civilization',
 'west',
 'philosophy'